## Date Arithmetic – Advanced Exercises

This notebook contains a series of **advanced (but not too advanced)** exercises focusing on **date arithmetic** with the `datetime` module and `timedelta` objects.

Concepts practiced:
- `datetime.timedelta` construction and arithmetic
- Working with `date` vs `datetime`
- Normalizing and truncating date/time information
- Implementing small utilities around recurring dates, business days, and periods

Each exercise has:
- A **problem description** (markdown)
- A **solution** (code), written using clean, best-practice Python


---
### Exercise 1 – Days Between Two Date/Datetime Values

**Goal:** Implement a small helper that computes a whole-number day difference between arbitrary `date` or `datetime` values.

**Requirements:**
1. Implement a function:
   ```python
   def days_between(a, b) -> int:
       ...
   ```
   where `a` and `b` can each be either a `datetime.date` or a `datetime.datetime`.
2. The function should return a signed integer number of days `b - a`.
3. If a value is a `datetime.datetime`, **ignore the time part** by converting it to a `date` first.
4. If `a` and `b` are not instances of `date`/`datetime`, raise `TypeError`.
5. Demonstrate:
   - `date(2020, 1, 1)` to `date(2020, 1, 10)`
   - `datetime(2020, 1, 1, 23, 0)` to `datetime(2020, 1, 2, 1, 0)` (should be 1 day)
   - reversed order (negative result)

This exercise practices:
- Converting between `datetime` and `date`
- Using `timedelta` for differences
- Validating inputs and handling types carefully


In [1]:
from __future__ import annotations
import datetime as _dt
from typing import Union

DateLike = Union[_dt.date, _dt.datetime]


def _to_date(value: DateLike) -> _dt.date:
    """Convert a date or datetime to a date.

    If `value` is already a date, return as-is.
    If it is a datetime, drop the time component.
    """
    if isinstance(value, _dt.datetime):
        return value.date()
    if isinstance(value, _dt.date):
        return value
    raise TypeError(f"Expected date or datetime, got {type(value)!r}")


def days_between(a: DateLike, b: DateLike) -> int:
    """Return number of whole days between `a` and `b` (b - a).

    Both `a` and `b` may be either date or datetime instances. Any time
    component is ignored by treating values as dates.
    """
    da = _to_date(a)
    db = _to_date(b)
    delta: _dt.timedelta = db - da
    return delta.days


if __name__ == "__main__":
    d1 = _dt.date(2020, 1, 1)
    d2 = _dt.date(2020, 1, 10)
    print("date -> date:", days_between(d1, d2))  # 9

    dt1 = _dt.datetime(2020, 1, 1, 23, 0)
    dt2 = _dt.datetime(2020, 1, 2, 1, 0)
    print("datetime -> datetime (ignoring time):", days_between(dt1, dt2))  # 1

    print("reversed order:", days_between(d2, d1))  # -9


date -> date: 9
datetime -> datetime (ignoring time): 1
reversed order: -9


---
### Exercise 2 – Generating Recurring Dates with `timedelta`

**Goal:** Implement a generator of recurring dates at a fixed interval.

**Requirements:**
1. Implement a function:
   ```python
   def iter_recurring_dates(start: datetime.date, *, step_days: int, count: int):
       ...
   ```
2. The function should:
   - Yield `count` dates starting from `start`.
   - Use a `timedelta(days=step_days)` as the step.
3. Validate that `step_days` is non-zero and `count` is non-negative; otherwise raise `ValueError`.
4. Demonstrate:
   - A weekly recurrence (step_days=7, count=4)
   - A negative step (e.g. going backwards in time)

This exercise practices:
- Using `timedelta` as a step
- Implementing simple, lazy generators over date ranges
- Basic validation and error handling


In [2]:
import datetime as _dt
from collections.abc import Iterator


def iter_recurring_dates(
    start: _dt.date,
    *,
    step_days: int,
    count: int,
) -> Iterator[_dt.date]:
    """Yield `count` dates spaced by `step_days` days starting at `start`.

    Parameters
    ----------
    start : datetime.date
        First date to yield.
    step_days : int
        Number of days between consecutive dates (may be negative).
    count : int
        Number of dates to generate (must be >= 0).
    """
    if step_days == 0:
        raise ValueError("step_days must be non-zero")
    if count < 0:
        raise ValueError("count must be non-negative")

    step = _dt.timedelta(days=step_days)
    current = start

    for _ in range(count):
        yield current
        current += step


if __name__ == "__main__":
    start_date = _dt.date(2023, 1, 1)
    print("Weekly recurrence:")
    for d in iter_recurring_dates(start_date, step_days=7, count=4):
        print("  ", d)

    print("\nGoing backwards (step_days=-3):")
    for d in iter_recurring_dates(start_date, step_days=-3, count=4):
        print("  ", d)


Weekly recurrence:
   2023-01-01
   2023-01-08
   2023-01-15
   2023-01-22

Going backwards (step_days=-3):
   2023-01-01
   2022-12-29
   2022-12-26
   2022-12-23


---
### Exercise 3 – Adding Business Days (Skipping Weekends)

**Goal:** Implement a function that adds business days to a `date`, skipping Saturdays and Sundays.

**Requirements:**
1. Implement a function:
   ```python
   def add_business_days(d: datetime.date, offset: int) -> datetime.date:
       ...
   ```
2. `offset` can be positive or negative:
   - Positive: move forward in time.
   - Negative: move backward in time.
3. Skip weekends (Saturday and Sunday; weekday indices 5 and 6).
4. Keep the logic simple and readable, not micro-optimized.
5. Demonstrate:
   - Adding 3 business days from a Wednesday.
   - Adding 3 business days from a Friday (crossing the weekend).
   - Subtracting business days (negative offset).

This exercise practices:
- `timedelta(days=1)` stepping
- Conditional logic based on `date.weekday()`
- Handling positive and negative offsets cleanly


In [3]:
import datetime as _dt


def add_business_days(d: _dt.date, offset: int) -> _dt.date:
    """Return the date that is `offset` business days from `d`.

    Business days are Monday–Friday. Weekends are skipped.
    `offset` may be positive or negative.
    """
    if offset == 0:
        return d

    step = 1 if offset > 0 else -1
    remaining = abs(offset)
    current = d
    one_day = _dt.timedelta(days=step)

    while remaining > 0:
        current += one_day
        if current.weekday() < 5:  # 0=Mon, ..., 4=Fri
            remaining -= 1

    return current


if __name__ == "__main__":
    wednesday = _dt.date(2023, 1, 4)  # Wednesday
    friday = _dt.date(2023, 1, 6)     # Friday

    print("Wednesday + 3 business days:", add_business_days(wednesday, 3))
    print("Friday + 3 business days:   ", add_business_days(friday, 3))
    print("Friday - 2 business days:   ", add_business_days(friday, -2))


Wednesday + 3 business days: 2023-01-09
Friday + 3 business days:    2023-01-11
Friday - 2 business days:    2023-01-04


---
### Exercise 4 – Normalizing Datetimes to Date Period Boundaries

**Goal:** Implement a helper to compute the **start** and **end** datetimes of a day given an arbitrary `datetime`.

**Requirements:**
1. Implement a function:
   ```python
   def day_bounds(dt: datetime.datetime) -> tuple[datetime.datetime, datetime.datetime]:
       ...
   ```
2. It should return a tuple `(start, end)` such that:
   - `start` is `00:00:00` of that day.
   - `end` is `23:59:59.999999` of that day.
3. Preserve `tzinfo` from `dt` on both `start` and `end`.
4. Demonstrate using:
   - A naive datetime.
   - An aware datetime in some fixed offset timezone (`timezone.utc` is fine).

This exercise practices:
- Using `replace` on `datetime`
- Reasoning about inclusive day ranges
- Preserving timezone information (`tzinfo`)


In [4]:
import datetime as _dt
from typing import Tuple


def day_bounds(dt: _dt.datetime) -> Tuple[_dt.datetime, _dt.datetime]:
    """Return the start and end datetime of the calendar day containing `dt`.

    Start is at 00:00:00, end is at 23:59:59.999999. tzinfo is preserved.
    """
    start = dt.replace(hour=0, minute=0, second=0, microsecond=0)
    end = dt.replace(hour=23, minute=59, second=59, microsecond=999_999)
    return start, end


if __name__ == "__main__":
    naive = _dt.datetime(2023, 1, 15, 13, 45, 10, 123456)
    start_naive, end_naive = day_bounds(naive)
    print("Naive:")
    print("  start:", start_naive)
    print("  end:  ", end_naive)

    aware = _dt.datetime(2023, 1, 15, 13, 45, tzinfo=_dt.timezone.utc)
    start_aware, end_aware = day_bounds(aware)
    print("\nAware (UTC):")
    print("  start:", start_aware, "tzinfo=", start_aware.tzinfo)
    print("  end:  ", end_aware, "tzinfo=", end_aware.tzinfo)


Naive:
  start: 2023-01-15 00:00:00
  end:   2023-01-15 23:59:59.999999

Aware (UTC):
  start: 2023-01-15 00:00:00+00:00 tzinfo= UTC
  end:   2023-01-15 23:59:59.999999+00:00 tzinfo= UTC


---
### Exercise 5 – Pretty-Printing Durations from `timedelta`

**Goal:** Implement a function that converts a `timedelta` into a human-friendly string.

**Requirements:**
1. Implement a function:
   ```python
   def format_timedelta(td: datetime.timedelta) -> str:
       ...
   ```
2. It should produce strings like:
   - `"2 days, 03:04:05"`
   - `"-1 day, 00:00:01"` (for negative timedeltas)
3. Rules:
   - Use absolute value of `td` for the breakdown, but prefix with `-` if `td` is negative.
   - Always show hours:minutes:seconds as zero-padded `HH:MM:SS`.
   - Include days part only if non-zero.
4. Demonstrate with:
   - Positive durations with and without days.
   - Negative durations.

This exercise practices:
- Understanding how `timedelta` represents days/seconds
- Basic integer arithmetic and string formatting
- Handling signs carefully for durations


In [5]:
import datetime as _dt


def format_timedelta(td: _dt.timedelta) -> str:
    """Format a timedelta as 'D days, HH:MM:SS' or 'HH:MM:SS'.

    Negative timedeltas are prefixed with '-'.
    """
    negative = td.total_seconds() < 0
    td_abs = -td if negative else td

    days = td_abs.days
    seconds = td_abs.seconds

    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)

    time_part = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    if days:
        result = f"{days} day{'s' if days != 1 else ''}, {time_part}"
    else:
        result = time_part

    return f"-{result}" if negative else result


if __name__ == "__main__":
    durations = [
        _dt.timedelta(hours=3, minutes=4, seconds=5),
        _dt.timedelta(days=2, hours=3, minutes=4, seconds=5),
        _dt.timedelta(days=-1, seconds=1),
        _dt.timedelta(days=-2, hours=5),
    ]

    for d in durations:
        print(d, "->", format_timedelta(d))


3:04:05 -> 03:04:05
2 days, 3:04:05 -> 2 days, 03:04:05
-1 day, 0:00:01 -> -23:59:59
-2 days, 5:00:00 -> -1 day, 19:00:00
